https://www.kaggle.com/yag320/list-of-fake-samples-and-public-private-lb-split

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook as tqdm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'FI.png', 'train.csv', 'sample_submission.csv']


In [3]:
test_path = '../input/test.csv'

df_test = pd.read_csv(test_path)
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

In [33]:
unique_samples = []
unique_count = np.zeros_like(df_test) #equal to np.zeros(df_test.shape)

In [34]:
print(df_test.shape)
print(unique_count.shape)
print(df_test.shape[1])

(200000, 200)
(200000, 200)
200


In [17]:
#np.unique
test_index = 0
print(np.unique(df_test[:,test_index]))
print(np.unique(df_test[:,test_index]).shape)

[ 0.1887  0.922   0.9302 ... 20.0649 20.7879 22.3234]
(65580,)


In [16]:
#np.unique with count and index
print(np.unique(df_test[:,test_index], return_counts=True, return_index=True))
print(np.unique(df_test[:,test_index], return_counts=True, return_index=True)[0].shape)
print(np.unique(df_test[:,test_index], return_counts=True, return_index=True)[1].shape)
print(np.unique(df_test[:,test_index], return_counts=True, return_index=True)[2].shape)

(array([ 0.1887,  0.922 ,  0.9302, ..., 20.0649, 20.7879, 22.3234]), array([ 14030, 119481, 194633, ...,    148,  71176,  12478]), array([2, 2, 1, ..., 3, 1, 3]))
(65580,)
(65580,)
(65580,)


In [35]:
for feature in tqdm(range(df_test.shape[1])):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] += 1

#np.countの返り値の１つ目は使用しないため、_という変数名で使用しないことを明示している
#index_, count_はindex, countとは異なる変数であることを明示するため、末尾にアンダースコアをつけている

In [36]:
unique_count.sum(axis=1)[0:10]

array([ 0.,  0.,  0., 40.,  0.,  0.,  0., 31.,  0.,  0.])

In [37]:
# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

#np.argwhere returns no zero indexes

In [38]:
print(len(real_samples_indexes))
print(len(synthetic_samples_indexes))

100000
100000


In [40]:
df_test_real = df_test[real_samples_indexes].copy()
print(df_test_real.shape)

(100000, 200)


In [41]:
generator_for_each_synthetic_sample = []

In [42]:
# Using 20,000 samples should be enough. 
# You can use all of the 100,000 and get the same results (but 5 times slower)
for cur_sample_index in tqdm(synthetic_samples_indexes[:20000]):
    cur_synthetic_sample = df_test[cur_sample_index]
    potential_generators = df_test_real == cur_synthetic_sample

    # A verified generator for a synthetic sample is achieved
    # only if the value of a feature appears only once in the
    # entire real samples set
    features_mask = np.sum(potential_generators, axis=0) == 1
    verified_generators_mask = np.any(potential_generators[:, features_mask], axis=1)
    verified_generators_for_sample = real_samples_indexes[np.argwhere(verified_generators_mask)[:, 0]]
    generator_for_each_synthetic_sample.append(set(verified_generators_for_sample))

In [43]:
public_LB = generator_for_each_synthetic_sample[0]
for x in tqdm(generator_for_each_synthetic_sample):
    if public_LB.intersection(x):
        public_LB = public_LB.union(x)

private_LB = generator_for_each_synthetic_sample[1]
for x in tqdm(generator_for_each_synthetic_sample):
    if private_LB.intersection(x):
        private_LB = private_LB.union(x)
        
print(len(public_LB))
print(len(private_LB))


50000
50000


In [44]:
np.save('public_LB', list(public_LB))
np.save('private_LB', list(private_LB))
np.save('synthetic_samples_indexes', list(synthetic_samples_indexes))